<a href="https://colab.research.google.com/github/Aaradhy23/projects/blob/main/find_your_movie_semantic_search.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [47]:
!pip install opendatasets sentence-transformers langchain-community chromadb --quiet #sentence transformers for open source application

In [48]:
import opendatasets as od
od.download("https://www.kaggle.com/datasets/akshaypawar7/millions-of-movies")

Please provide your Kaggle credentials to download this dataset. Learn more: http://bit.ly/kaggle-creds
Your Kaggle username: aaradhy024
Your Kaggle Key: ··········
Dataset URL: https://www.kaggle.com/datasets/akshaypawar7/millions-of-movies


100%|██████████| 164M/164M [00:09<00:00, 17.4MB/s]


In [3]:
import pandas as pd
from langchain.document_loaders.csv_loader import CSVLoader
from langchain_community.vectorstores import Chroma
from langchain_community.document_loaders import DataFrameLoader
from langchain.embeddings import HuggingFaceEmbeddings


In [49]:
data_df= pd.read_csv("/content/millions-of-movies/movies.csv")[["title","overview"]]
data_df.head()

,title,overview
0,Godzilla x Kong: The New Empire,Following their explosive showdown Godzilla an...
1,Meg 2: The Trench,An exploratory dive into the deepest depths of...
2,The Pope's Exorcist,Father Gabriele Amorth Chief Exorcist of the V...
3,Transformers: Rise of the Beasts,When a new threat capable of destroying the en...
4,Dune: Part Two,Follow the mythic journey of Paul Atreides as ...


In [50]:
#clean data
print(data_df.shape)
print()
print(data_df.isna().sum())

(722415, 2)

title            6
overview    118306
dtype: int64


In [51]:
Embeddings= HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2",model_kwargs={'device':'cuda'})

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [52]:
loader= DataFrameLoader(data_df,page_content_column="overview")#here page content column is decided according to our database
descriptions= loader.load()
#description object creation

In [10]:
CHROMA_PATH= "chroma_movies_db"
chroma_db= Chroma.from_documents(
    descriptions,Embeddings, persist_directory=CHROMA_PATH
)

In [53]:
chroma_db#here we dont need to divid into chunks as we directly have the answer in dataset

In [54]:
del chroma_db

In [55]:
chroma_db=Chroma(persist_directory="/content/chroma_movies_db",embedding_function=Embeddings)
chroma_db

In [60]:
query="""
Following their explosive showdown Godzilla and Kong must reunite against a colossal
"""

relevant_docs= chroma_db.similarity_search(query,k=10)#k determines closest entries

for doc in relevant_docs:
  print(doc.metadata["title"])
  print(doc.page_content)
  print()

Kong: King of the Apes
In 2050, an evil scientist is about to unleash an army of robotic dinosaurs. Now it's up to Kong and friends to fight back and save the day.

GODZILLA City on the Edge of Battle
On Earth, Haruo and his companions make contact with the indigenous Houtua tribe. To defeat Godzilla, they set out for the old Mechagodzilla facility.

Godzilla
A desperate group of refugees attempts to recolonize Earth 20,000 years after Godzilla took over. But one young man wants revenge above all else.

Tremors: Shrieker Island
When massive mutant creatures infiltrate an island nature preserve, scientists recruit a legendary monster hunter to battle the beasts.

GANTZ:O
Teams of recently deceased people who've been revived and given high-tech weapons must cooperate to defeat an army of monsters in Tokyo and Osaka.

InuYasha: The Movie 2: The Castle Beyond the Looking Glass
With their biggest foe seemingly defeated, InuYasha and his friends return to everyday life. But the peace is soon